<a href="https://colab.research.google.com/github/gopalm-ai/Quantum_Machine_Learning_with_Python/blob/main/Simon_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Simons Algorithm

In [2]:
! pip3 install cirq

In [3]:
import numpy as np
import cirq

In [4]:
def oracle(input_qubits, target_qubits, circuit):
  # Oracle for Secret Code 110
  circuit.append(cirq.CNOT(input_qubits[2], target_qubits[1]))
  circuit.append(cirq.X(target_qubits[0]))
  circuit.append(cirq.CNOT(input_qubits[2], target_qubits[0]))
  circuit.append(cirq.CCNOT(input_qubits[0], input_qubits[1], target_qubits[0]))
  circuit.append(cirq.X(input_qubits[0]))
  circuit.append(cirq.X(input_qubits[1]))
  circuit.append(cirq.CCNOT(input_qubits[0], input_qubits[1], target_qubits[0]))
  circuit.append(cirq.X(input_qubits[0]))
  circuit.append(cirq.X(input_qubits[1]))
  circuit.append(cirq.X(target_qubits[0]))
  return circuit

In [7]:
def simons_algorithm_circuit(num_qubits=3, copies=1000):
  """
  Build the circuit for Simon's Algorithm
  :param num_qubits
  :return: cirq circuit
  """
  input_qubits = [cirq.LineQubit(i) for i in range(num_qubits)]
  target_qubits = [cirq.LineQubit(k) for k in range(num_qubits, 2 * num_qubits)]
  circuit = cirq.Circuit()
  # Create Equal Superposition state for the
  # Input qubits through Hadamard Transform
  circuit.append([cirq.H(input_qubits[i]) for i in range(num_qubits)])
  # Pass the Superposition state through the oracle
  circuit = oracle(input_qubits, target_qubits, circuit)
  # Apply Hadamard transform on the input corners
  circuit.append([cirq.H(input_qubits[i]) for i in range(num_qubits)])
  # Measure the input and the target qubits
  circuit.append(cirq.measure(*(input_qubits + target_qubits), key='Z'))
  print("Circuit diagram for Simon's Algorithm follows")
  print(circuit)
  # Simulate algorithm
  sim = cirq.Simulator()
  result = sim.run(circuit, repetitions=copies)
  out = dict(result.histogram(key='Z'))
  out_result = {}
  for k in out.keys():
    new_key = "{0:b}".format(k)
    if len(new_key) < 2*num_qubits:
      new_key = (2*num_qubits - len(new_key))*'0' + new_key
    out_result[new_key] = out[k]
  print(out_result)

In [8]:
if __name__ == '__main__':
  simons_algorithm_circuit()

Circuit diagram for Simon's Algorithm follows
                  ┌──┐
0: ───H────────────@─────X───@───X───H───M('Z')───
                   │         │           │
1: ───H────────────@─────X───@───X───H───M────────
                   │         │           │
2: ───H───@───@────┼H────────┼───────────M────────
          │   │    │         │           │
3: ───X───┼───X────X─────────X───X───────M────────
          │                              │
4: ───────X──────────────────────────────M────────
                                         │
5: ──────────────────────────────────────M────────
                  └──┘
{'000010': 69, '110000': 60, '001100': 56, '000100': 56, '001110': 52, '001010': 65, '110110': 63, '001000': 46, '111010': 72, '000110': 64, '111000': 65, '111100': 64, '110100': 65, '000000': 65, '111110': 73, '110010': 65}
